In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = Conv2D(32, (3, 3), activation='relu')
        self.pool1 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(64, activation='relu')
        self.dense2 = Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        output = self.dense2(x)
        return output



# Define ImageDataGenerator for normalizing pixel values
normalize_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
normalize_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load train data using ImageDataGenerator with shuffling
train_data = normalize_train.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\train",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary',  # Set class_mode to 'binary' for binary classification
    shuffle=True
)

# Load test data using ImageDataGenerator with shuffling
test_data = normalize_test.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\test",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary',  # Set class_mode to 'binary' for binary classification
    shuffle=True
)

# Instantiate the model
model = CustomModel()

# Compile the model
model.compile(optimizer='adam',
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

# Train the model
model.fit(train_data, epochs=5, validation_data=test_data)

# Evaluate the model
model.evaluate(test_data)


Found 20 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


Epoch 1/5
2/2 [==============================] - 1s 358ms/step - loss: 22.6988 - binary_accuracy: 0.3500 - val_loss: 14.0880 - val_binary_accuracy: 0.5000
Epoch 2/5
2/2 [==============================] - 0s 337ms/step - loss: 4.5126 - binary_accuracy: 0.5000 - val_loss: 1.1604 - val_binary_accuracy: 0.5000
Epoch 3/5
2/2 [==============================] - 0s 202ms/step - loss: 0.7062 - binary_accuracy: 0.5000 - val_loss: 0.6931 - val_binary_accuracy: 0.5000
Epoch 4/5
2/2 [==============================] - 0s 180ms/step - loss: 0.6931 - binary_accuracy: 0.5000 - val_loss: 0.8878 - val_binary_accuracy: 0.5000
Epoch 5/5
1/1 [==============================] - 0s 66ms/step - loss: 0.6931 - binary_accuracy: 0.5000


[0.6931478381156921, 0.5]

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

class CustomConv2D(Layer):
    def __init__(self, filters, kernel_size, activation='relu'):
        super(CustomConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
                                      initializer='glorot_uniform',
                                      trainable=True,
                                      name='kernel')
        self.bias = self.add_weight(shape=(self.filters,),
                                    initializer='zeros',
                                    trainable=True,
                                    name='bias')
        # super(CustomConv2D, self).build(input_shape)

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output

class CustomDense(Layer):
    def __init__(self, units, activation='relu'):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='glorot_uniform',
                                      trainable=True,
                                      name='kernel')
        self.bias = self.add_weight(shape=(self.units,),
                                    initializer='zeros',
                                    trainable=True,
                                    name='bias')
        # super(CustomDense, self).build(input_shape)

    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel)
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output

class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = CustomConv2D(32, (3, 3), activation='relu')
        self.pool1 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.dense1 = CustomDense(64, activation='relu')
        self.dense2 = CustomDense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        output = self.dense2(x)
        return output

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            loss = self.compiled_loss(y, y_pred)  # Compute loss

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

# Define ImageDataGenerator for normalizing pixel values
normalize_train = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
normalize_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load train data using ImageDataGenerator with shuffling
train_data = normalize_train.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\train",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary',  # Set class_mode to 'binary' for binary classification
    shuffle=True
)

# Load test data using ImageDataGenerator with shuffling
test_data = normalize_test.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\test",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary',  # Set class_mode to 'binary' for binary classification
    shuffle=True
)

# Instantiate the model
model = CustomModel()

# Compile the model
model.compile(optimizer='adam',
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

# Train the model
model.fit(train_data, epochs=5, validation_data=test_data)

# Evaluate the model
model.evaluate(test_data)



Found 20 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


Epoch 1/5
2/2 [==============================] - 1s 319ms/step - loss: 17.6061 - binary_accuracy: 0.1500 - val_loss: 1.0380 - val_binary_accuracy: 0.5000
Epoch 2/5
2/2 [==============================] - 0s 151ms/step - loss: 2.1725 - binary_accuracy: 0.5000 - val_loss: 0.5490 - val_binary_accuracy: 0.6667
Epoch 3/5
2/2 [==============================] - 0s 146ms/step - loss: 0.3904 - binary_accuracy: 0.7500 - val_loss: 0.1460 - val_binary_accuracy: 1.0000
Epoch 4/5
2/2 [==============================] - 0s 145ms/step - loss: 0.1938 - binary_accuracy: 0.9000 - val_loss: 0.1474 - val_binary_accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 62ms/step - loss: 0.5406 - binary_accuracy: 0.6667


[0.5406013131141663, 0.6666666865348816]